#  <span style="color:darkblue"> Optimization </span>

<hr style="border:6px solid black"> </hr>

## <span style="color:darkblue"> Learning Objectives </span>

- Classify optimization problems as linear, quadratic, nonlinear, and mixed-integer problems.
- Learn how to interpret the solutions of optimization problems.
- Learn how to input optimization problems in Julia/JuMP).
- Assess the sensitivity of a solution with respect to constraints.
- Learn how to formulate and solve optimization problems for multiple examples.

<hr style="border:6px solid black"> </hr>

## <span style="color:darkblue"> Organization </span>

- First, we'll review some of the types of optimization problems.
- Next, we'll solve a Sudoku puzzle.
- You'll formulate and solve a simple blending and pooling problem.
- Lastly, we'll analyze the blending and pooling problem from a sensitivity perspective.
    
<hr style="border:6px solid black"> </hr>

## <span style="color:darkblue"> Problem Forms </span>

Optimizers generally exploit a specific mathematical structure in the problem formulation as a means to solve a problem. In fact, some underlying assumptions are always required to ensure a problem is optimized. One common example arrise from combining KKT conditions with constraint qualifications or alternatively regularity conditions for nonlinear programs.

These problem types range from linear programs for which problems with 100k variables and constraints may routinely be solved [in less than a minute](http://plato.asu.edu/ftp/lpsimp.html) to continuous convex problems to nonconvex nonlinear mixed-integer problems (were some problems with fewer than 5 variables have yet to be solved). As such, using an appropriate optimizer for a particular problem type can transform an unsolveable problem into a trival one. The most heavily studied and widely used descriptors for each program are linear, quadratic, nonlinear, and mixed-integer.

**Types of based on objectives/constraints:**

- **Linear:** All the constraints and the objective are linear.
- **Quadratic** all the constraints and the objective are quadratic or linear.
- **Nonlinear** at least one constraint or the objective is not linear.

**Types based on manner of variable used:**

- **Continuous:** All variables can vary between values in some possibly open interval.
- **Integer:** All variables are integer valued.
- **Mixed-integer:** The problem contains both integer and continuous variables.

One of the central focuses of optimization research has focused on developing specialized routines for solving programs with other special forms. A number of other important forms include [specially-ordered sets](https://link.springer.com/article/10.1007/BF01589393), [second-order cones](https://ieeexplore.ieee.org/abstract/document/6983691), [equilibrium constraints](https://faculty.wcas.northwestern.edu/~lchrist/Ferris_mathematical_programs2.pdf), as well as a myriad of special nonlinear forms. When attempting to solve a difficult model it's worth checking a few references (such as [Mosek's Modeling Cookbook](https://docs.mosek.com/MOSEKModelingCookbook-letter.pdf)) to see if the problem can be re-written as simplier form or a simplier problem type.

This trend has lead to the development of a number of specialized languages and software packages used to describe optimization problems interface with a myriad of different optimizers. In Python, the main tools for this are [Pyomo]() and [CVXOPT](https://cvxopt.org/). In Julia, we have [JuMP](https://jump.dev/JuMP.jl/stable/), and [Convex.jl](https://jump.dev/Convex.jl/stable/). A number of other commercial offerings available include [GAMS](https://www.gams.com/), [AIMMS](https://en.wikipedia.org/wiki/AIMMS), and [AMPL](https://ampl.com/) along with a number of offerings associated with commercially available solvers such as [CPLEX Optimization Studio by IBM](https://www.ibm.com/analytics/cplex-optimizer) or [Xpress Mosel for FICO Express](https://www.fico.com/en/products/fico-xpress-optimization). We'll work through formulating and solving a couple of optimization problems using JuMP in this workbook.

<hr style="border:2px solid gray"> </hr>

In [1]:
# Run this block once. Adds Julia packages to your Julia local installation from the package manager

using Pkg               # Import functions from the package manager into this session.
Pkg.add("JuMP")         # A modeling language for Mathematical optimization in Julia.
Pkg.add("Ipopt")        # A highly performant nonlinear optimizer.
Pkg.add("GLPK")         # An open-source linear and mixed-integer linear optimizer

   Updating registry at `C:\Users\wilhe\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\wilhe\Project.toml`
No Changes to `C:\Users\wilhe\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\wilhe\Project.toml`
No Changes to `C:\Users\wilhe\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\wilhe\Project.toml`
No Changes to `C:\Users\wilhe\Manifest.toml`


Let's start with a very basic problem so we can develop some familarity with the JuMP modeling language. We'll begin by solving a Sudoku puzzle using optimization software.

<hr style="border:2px solid gray"> </hr>

## <span style="color:darkblue">  Solving Sudoku with Mixed-Integer Programming* </span>

* Adapted from a now defunct example by Ian Dunning at https://www.juliaopt.org/notebooks/JuMP-Sudoku.html.

Sudoku is a popular number puzzle. The goal is to place the digits 1,...,9 on a nine-by-nine grid, with some of the digits already filled in. Your solution must satisfy the following rules:

    The numbers 1 to 9 must appear in each 3x3 square
    The numbers 1 to 9 must appear in each row
    The numbers 1 to 9 must appear in each column

Any combination satisfying the above rules is a valid solution. Thus, we're simply seeking any feasible solution. While this isn't strictly an optimization problem (it's a constraint satisfaction problem), but we can make it into equivalent optimization problem. Namely, we can just specify that the objective is some constant value of our choosing. 

We can model this problem using 0-1 integer programming: a problem where all the decision variables are binary. We'll use JuMP to create the model, and then we can solve it with any integer optimizer.

In [2]:
using JuMP, GLPK                    # import functions from the JuMP & GLPK package into your current environment

# create a model called "sudoku" which holds all variables, constraints, 
# functions, and options needed to solve the sudoku problem
sudoku = Model(GLPK.Optimizer)

# add binary variables
@variable(sudoku, x[i=1:9, j=1:9, k=1:9], Bin)   # Bin indicates the variable is binary (has a value of 0 or 1)

9×9×9 Array{VariableRef,3}:
[:, :, 1] =
 x[1,1,1]  x[1,2,1]  x[1,3,1]  x[1,4,1]  …  x[1,7,1]  x[1,8,1]  x[1,9,1]
 x[2,1,1]  x[2,2,1]  x[2,3,1]  x[2,4,1]     x[2,7,1]  x[2,8,1]  x[2,9,1]
 x[3,1,1]  x[3,2,1]  x[3,3,1]  x[3,4,1]     x[3,7,1]  x[3,8,1]  x[3,9,1]
 x[4,1,1]  x[4,2,1]  x[4,3,1]  x[4,4,1]     x[4,7,1]  x[4,8,1]  x[4,9,1]
 x[5,1,1]  x[5,2,1]  x[5,3,1]  x[5,4,1]     x[5,7,1]  x[5,8,1]  x[5,9,1]
 x[6,1,1]  x[6,2,1]  x[6,3,1]  x[6,4,1]  …  x[6,7,1]  x[6,8,1]  x[6,9,1]
 x[7,1,1]  x[7,2,1]  x[7,3,1]  x[7,4,1]     x[7,7,1]  x[7,8,1]  x[7,9,1]
 x[8,1,1]  x[8,2,1]  x[8,3,1]  x[8,4,1]     x[8,7,1]  x[8,8,1]  x[8,9,1]
 x[9,1,1]  x[9,2,1]  x[9,3,1]  x[9,4,1]     x[9,7,1]  x[9,8,1]  x[9,9,1]

[:, :, 2] =
 x[1,1,2]  x[1,2,2]  x[1,3,2]  x[1,4,2]  …  x[1,7,2]  x[1,8,2]  x[1,9,2]
 x[2,1,2]  x[2,2,2]  x[2,3,2]  x[2,4,2]     x[2,7,2]  x[2,8,2]  x[2,9,2]
 x[3,1,2]  x[3,2,2]  x[3,3,2]  x[3,4,2]     x[3,7,2]  x[3,8,2]  x[3,9,2]
 x[4,1,2]  x[4,2,2]  x[4,3,2]  x[4,4,2]     x[4,7,2]  x[4,8,2]  x[4,9,2

Only one digit can be in each square. So for each square represented by a pair of i,j variables the sum over k should be equal to one.

In [3]:
for i = 1:9, j = 1:9  
    @constraint(sudoku, sum(x[i,j,k] for k=1:9) == 1)
end

Each variable can only appear once in each column.

In [4]:
for j = 1:9, k = 1:9
    @constraint(sudoku, sum(x[i,j,k] for i=1:9} == 1)
end

LoadError: syntax: unexpected "}"

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Each variable can only appear once in each row. Fill in the appropriate expression.
</div>

In [5]:
for ind = 1:9, k = 1:9
    # FILL IN THE CODE HERE
end

Each variable can only appear once in each 3-by-3 subgrid.

In [6]:
# i is the top left row, j is the top left column
# We'll sum from i to i+2, e.g. i=4, r=4, 5, 6
for i = 1:3:7, j = 1:3:7, k = 1:9
    @constraint(sudoku, sum(x[i,ind,k] for r=i:i+2, c=j:j+2) == 1)
end

LoadError: syntax: unexpected "]"

Fills in the provided values using zero to represent an empty cell

In [7]:
# enter the initial values
init_val = [ 5 3 0 0 7 0 0 0 0;
             6 0 0 1 9 5 0 0 0;
             0 9 8 0 0 0 0 6 0;
             8 0 0 0 6 0 0 0 3;
             4 0 0 8 0 3 0 0 1;
             7 0 0 0 2 0 0 0 6;
             0 6 0 0 0 0 2 8 0;
             0 0 0 4 1 9 0 0 5;
             0 0 0 0 8 0 0 7 9]

# if the value is specified initially add a constraint fixing the variables to that value.
for i = 1:9, j = 1:9
    if init_val[i,j] != 0
        @constraint(sudoku, x[i,j,init_val[i,j]] == 1)
    end
end

LoadError: UndefVarError: init_sol not defined

<div class="alert alert-block alert-info">
<b>What type of optimization problem was the sudoku? Is GLPK an appropriate optimizer for this type of problem?</b> You'll want to search for GLPK's documentation and check there.
</div>

<hr style="border:2px solid gray"> </hr>

In [8]:
optimize!(m)

x_val = value(x) # Extract the values of x

LoadError: UndefVarError: m not defined

To display the solution, we need to look for the values of **x**[i, j, k] that are equal to 1.

<div class="alert alert-block alert-warning">
<b>INTERACTIVE! Unpack the solution to a 9-by-9 matrix and confirm this is a valid Sudoku solution.</b> Note that the 
solution returned may not be an integer. So you'll need to round this to the nearest integer using <b>iround(y)</b>. 
</div>

In [9]:
# FILL IN THE CODE HERE

<hr style="border:2px solid gray"> </hr>

## <span style="color:darkblue"> Integrating water system treatment in the process industry </span>

* Adapted from [R. Karuppiah & I. Grossmann. Global optimization for the synthesis of integrated water systems in chemical processes. Computers and Chemical Engineering 30 (2006) 650–673](https://www.sciencedirect.com/science/article/abs/pii/S0098135405002991)

The efficient utilitization of water will be an integral part to solving the numerous water management issues of today and the near future. The increasing usage of water, anticipated water scarities, and changing regulations are expected to be central challenge in the next decades. These challenges extend to the adaptation of numerous industrial processes.
as fresh water is a key resource used a number of industrial processes. These include the washing operations in the food processing, the thermal processing (cooling, quenching, and scrubbing) in iron and steel manufacturing, and in the desalination of crude oil. In each of this processes, some level of contamination is introduced into the freshwater by the process. This process water is then treated in a central facility to remove contaminants and satisfy local regulatory specifications for the disposal of wastewater. In design or improving this central facility, two central questions must be considered. 1) What unit operations need to be included? 2) How should these process units be connected?

A typical design objective for a wastewater treatment plant may consist of minimize it's overall cost in a given year of operation. That is we minimize the sum of the cost fo freshwater intake, the investment cost associated with each treatment unit, and the operating cost of each unit. This can be achieved by minimizing the follow objective:

$\Phi = A_R\sum_{j} C_j^{Inv} F_j^{\alpha} + H\sum_{j} F_j C_j^{Op} + H F_{fw} C_{fw}$ 

where H is the number of hours of operation per annum ($h$); the cost of freshwater is given by $C_{fw}$ (`$/ton`); the fresh water intake mass if given by $F_{fw}$ (`ton/h`); The annualized factor for investment on treatment units is given by $A_R$; the $C_j^{Inv} F_j^{\alpha}$ is the investment cost (`$`) of the $j$-th treatment unit which treats a flowrate stream of contaminated water $F_j$ (`ton/h`); $C_j^{Op} F_j^{\alpha}$ is the operating cost (`$/h`) of the $j$-th treatment unit which treats a flowrate of contaminated water $F_j$ (`ton/h`); the parameter $\alpha$ is a constant cost function exponent $0 \leq \alpha \leq 1$.

The **mixers** units can be described as follows:
- The outlet flowrate is the sum of the inlet stream flow rate: $F^{outlet} = \sum_{i = 1}^{n} F^{inlet}_i $ where $F^{outlet}$, $F^{inlet}_1$, $\ldots$, $F^{inlet}_n$ are given in (tons/hr) and $n$ is the number of inlets streams.
- The mass of each contaminant in the outlet equals the mass of each contaminant in the inlet streams: $F^{outlet}C_j^{outlet} = \sum_{i = 1}^{n} F^{inlet}_i C_j^{inlet}$ 

The **splitter** units:
- The outlet flowrates are the sum of the inlet flowrate: $F^{inlet} = \sum_{i = 1}^{m} F^{outlet}_i $ where $F^{inlet}$, $F^{outlet}_1$, $\ldots$, $F^{outlet}_m$ are given in (tons/hr) and $m$ is the number of outlet streams.
- The each outlet streams contaminant concentration is equal to the inlet stream contaminant concentration: $F^{outlet}_m$. 

A fixed mass of contaminant is added to a stream by each **process unit**:
- A process unit has a single inlet stream and a single outlet stream: $F^{inlet} = F^{outlet}$
- The process unit reduces the l $F^{inlet}C_j^{inlet} + L^{p}_j = F^{outlet}C_j^{outlet}$

The **treatment units**:
- A process unit has a single inlet stream and a single outlet stream with the same flowrate: $F^{inlet} = F^{outlet}$
- The concentration of contaminant in the outlet $C_j^{outlet} = \beta^{t}_j C_j^{inlet}$ where $\beta^{t}_j$ is the removal ratio of $j$.

Two contaminants A & B are introduced into the system and the concentration of each contaminant must be reduced to 10ppm prior to discharging it into the enviroment. The cost of freshwater is assumed to be $1/ton, the annualized factor for investment on the treatment unit is taken to be 0.1, and the total time of operation of the plant is 8000h/year. 

<img src="wastewater.png" width="800">

The flowrate of water required by each process unit, along with containment inlet concentration limits, and amount of contaminant discharged are given in Table 1. We'll assume an $\alpha$ value of 0.7, an investment 

|                   | Flowrate (ton/h) |     Discharge A (kg/h)       |     Discharge B  (kg/h)       |     Max inlet concentration A (ppm)       |    Max inlet concentration  B  (ppm)      |
| -----------       |     -----------  | ----------- | ----------- | ----------- | ----------- |
| Process Unit #1   | 40               | 1           | 1.5         | 0           |  0          |
| Process Unit #2   | 50               | 1           | 1           | 50          | 50          |

The efficiency of the treatment units are given by the removal ratio in Table 2 along with cost model constants:

|                        | Removal Ratio of A | Removal Ratio of B | $C_j^{Inv}$        | $C_j^{Op}$         |
| -----------            |     -----------    | -----------        | -----------        | -----------        |
| Treatment Unit #1      | 95\%               |   0\%              |   16800            | 1                  |
| Treatment Unit #2      | 0\%                | 95\%               |   12600            | 0.0067             |


<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Is this model convex or nonconvex? Continuous or mixed-integer? Why?
</div>

<hr style="border:2px solid gray"> </hr>

In [1]:
using JuMP, Ipopt                    # import functions from the JuMP & Ipopt package into your current environment

wastewater = Model(Ipopt.Optimizer)  # create a model which stores an optimization problem
                                     # indicate that the optimizer Ipopt should be used to solve the model

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1278
┌ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1278


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Define the variable in the model for the flowrates (<b>Hint:</b> It may be easier to write concentrations and flowrates as vectors rather than a series of variables. Also, assume that the max flowrate in a system is about 5000 tons, at least 1 ton of wastewater was used). Also, assume the flowrate in each treatment unit is atleast 0.1 tons/hour. A flowrate of 0.0 may cause a numerical error in the objective function. Why do you think that would be?
</div>

<div class="alert alert-block alert-info">
<b>Comment:</b> Some optimizers require that all variables have both lower and upper bounds defined. If there aren't any obvious choices in a problem, you may have to exercise your best judgement and choose reasonable bounds.
</div>

In [2]:
max_flow = 5000.0 # max flowrate used in plant
max_conc = 1.00

@variable(wastewater, 0 <= fB <= max_flow)
@variable(wastewater, 0 <= c1B <= max_conc)
@variable(wastewater, 0 <= c2B <= max_conc)

max_p2_conc = 50/(1000)^2
@variable(wastewater, 0 <= fC <= max_flow)
@variable(wastewater, 0 <= c1C <= max_p2_conc)
@variable(wastewater, 0 <= c2C <= max_p2_conc)

@variable(wastewater, 0 <= fD <= max_flow)
@variable(wastewater, 0 <= c1D <= max_conc)
@variable(wastewater, 0 <= c2D <= max_conc)

@variable(wastewater, 0 <= fK <= max_flow)
@variable(wastewater, 0 <= c1K <= max_conc)
@variable(wastewater, 0 <= c2K <= max_conc)

# input variables
@variable(wastewater, 1.0 <= fA <= max_flow)
c1A = 0.0
c2A = 0.0

@variable(wastewater, 0.1 <= fE <= max_flow)
@variable(wastewater, 0 <= c1E <= max_conc)
@variable(wastewater, 0 <= c2E <= max_conc)

@variable(wastewater, 0 <= fF <= max_flow)
@variable(wastewater, 0 <= c1F <= max_conc)
@variable(wastewater, 0 <= c2F <= max_conc)

@variable(wastewater, 0.1 <= fG <= max_flow)
@variable(wastewater, 0 <= c1G <= max_conc)
@variable(wastewater, 0 <= c2G <= max_conc)

@variable(wastewater, 0 <= fH <= max_flow)
@variable(wastewater, 0 <= c1H <= max_conc)
@variable(wastewater, 0 <= c2H <= max_conc)

@variable(wastewater, 0 <= fI <= max_flow)
@variable(wastewater, 0 <= c1I <= max_conc)
@variable(wastewater, 0 <= c2I <= max_conc)

@variable(wastewater, 0 <= fJ <= max_flow)
@variable(wastewater, 0 <= c1J <= max_conc)
@variable(wastewater, 0 <= c2J <= max_conc)

@variable(wastewater, 0 <= fL <= max_flow)
@variable(wastewater, 0 <= c1L <= max_conc)
@variable(wastewater, 0 <= c2L <= max_conc)

outlet_spec = 10.0/(1000)^2   # 10ppm
@variable(wastewater, 0 <= fM <= max_flow)
@variable(wastewater, 0 <= c1M <= outlet_spec)
@variable(wastewater, 0 <= c2M <= outlet_spec)

c2M

Next we'll add constraints for the first mixer unit and the second process unit below:

In [3]:
# mixer #1
@constraint(wastewater, mixer1_flow, fB + fK - fC == 0)
@constraint(wastewater, mixer1_c1, c1B*fB + c1K*fK - c1C*fC == 0)
@constraint(wastewater, mixer1_c2, c2B*fB + c2K*fK - c2C*fC == 0)

# process unit #2
Lp12 = 1/1000
Lp22 = 1/1000
@constraint(wastewater, process2_flow, fC - fD == 0)
@constraint(wastewater, process2_c1, fC*c1C + Lp12 - fD*c1D == 0)
@constraint(wastewater, process2_c2, fD*c1C + Lp22 - fD*c2D == 0)

process2_c2 : fD*c1C - c2D*fD == -0.001

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Now we'll need to define constraints for the remaining mixing units, splitters, treatment, and process units.
</div>

In [4]:
# mixer #2
@constraint(wastewater, mixer2_flow, fL + fF - fG == 0)
@constraint(wastewater, mixer2_c1, c1L*fL + c1F*fF - c1G*fG == 0)
@constraint(wastewater, mixer2_c2, c2L*fL + c2F*fF - c2G*fG == 0)

# mixer #3
@constraint(wastewater, mixer3_flow, fH + fJ - fM == 0)
@constraint(wastewater, mixer3_c1, c1H*fH + c1J*fJ - c1M*fM == 0)
@constraint(wastewater, mixer3_c2, c2H*fH + c2J*fJ - c2M*fM == 0)

# splitter #1
@constraint(wastewater, splitter1_flow, fD - fE - fF == 0)
@constraint(wastewater, splitter1_c1a, c1D - c1E == 0)
@constraint(wastewater, splitter1_c1b, c1D - c1F == 0)
@constraint(wastewater, splitter1_c2a, c2D - c2E == 0)
@constraint(wastewater, splitter1_c2b, c2D - c2F == 0)

# splitter #2
@constraint(wastewater, splitter2_flow, fI - fJ - fK - fL == 0)
@constraint(wastewater, splitter2_c1a, c1I - c1J == 0)            # Concentration of component 1 in steam I
@constraint(wastewater, splitter2_c1b, c1I - c1K == 0)            # equals concentration in steam J
@constraint(wastewater, splitter2_c1c, c1I - c1L == 0)            # same for each input/output steeam
@constraint(wastewater, splitter2_c2a, c2I - c2J == 0)
@constraint(wastewater, splitter2_c2b, c2I - c2K == 0)
@constraint(wastewater, splitter2_c2c, c2I - c2L == 0)

# treatment unit #1
removal_ratio1 = 1.0 - 0.95
@constraint(wastewater, treatment1_flow, fE - fH == 0)
@constraint(wastewater, treatment1_c1, removal_ratio1*c1E - c1H == 0)
@constraint(wastewater, treatment1_c2, c2E - c2H == 0)

# treatment unit #2
removal_ratio2 = 1.0 - 0.95
@constraint(wastewater, treatment2_flow, fG - fI == 0)
@constraint(wastewater, treatment2_c1, c1G - c1I == 0)
@constraint(wastewater, treatment2_c2, removal_ratio2*c2G - c2I == 0)

# process unit #1
Lp11 = 1.5/1000
Lp21 = 1/1000
@constraint(wastewater, process1_flow, fA - fB == 0)
@constraint(wastewater, process1_c1, fA*c1A + Lp11 - fB*c1B == 0)
@constraint(wastewater, process1_c2, fA*c2A + Lp21 - fB*c2B == 0)

process1_c2 : -c2B*fB == -0.001

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Add an objective function to the model and check the help for `JuMP.@objective` and `JuMP.@NLobjective` (after running the above blocks) to check for the syntax needed. 
</div>

In [5]:
# financial assumptions
AR = 0.1       # annualized factor
H = 8000       # total hours plant operates
alpha = 0.7    # scaling term for capital cost
Cwater = 1.0   # cost of freshwater ($/ton)

# investment cost
Cinv1 = 16800.0
Cinv2 = 12600.0

# marginal operating cost
Cop1 = 1
Cop2 = 0.0067
@NLexpression(wastewater, capital_cost, AR*(Cinv1*fE^alpha + Cinv2*fG^alpha))
@NLexpression(wastewater, operating_cost, Cop1*fE + Cop2*fG + H*fA*Cwater)
@NLobjective(wastewater, Min, capital_cost + operating_cost)

Now, we simply optimize the model.

In [ ]:
JuMP.optimize!(wastewater)

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Next we extract information about the optimization problem. We'll want to check the status of the solve and the objective value initially. Generally, optimizers will return one or more status codes to indicate if the solve was successful and to indicate any potential numerical issues that occur. The raw form of these are specific to each optimizer but JuMP intrepets these into standard status codes. A termination status accessed by the function <b>termination_status</b> indicating the optimizer status and a primal status describing the meaning of the solution accessed by <b>primal_status</b>. Lastly, the objective value is accessed by <b>objective_value</b>. Check the help for descriptions of the <b>TerminationStatusCode</b> and <b>ResultStatusCode</b>. Are these what you'd expect?
</div>

In [ ]:
ww_termination_status = termination_status(wastewater)
ww_primal_status = primal_status(wastewater)
println("Termination Status is $(ww_termination_status) while primal status is $(ww_primal_status)")

Let's look into the **sensitivity** of the problems. That is to say, how much will changing a constraint and/or the variable bounds result in changing the solution? If we can perturb a constraint without affect this solution then the constraint is inactive. Otherwise, we can access the sensitivity via the **dual** the constraint $g(x) \leq a$ which is the change in the optimal value per change in $a$. Use **LowerBoundRef** and **UpperBoundRef** to get constraints corresponding to the box constraints on the variables.

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> How much effect would changing the 10ppm effluent specification slightly have on the process cost? <b> Hint: </b> Use the function <b> dual </b> to recover the dual value of the constraint defining this specification. Does this match the result you get by modifying the constraint and resolving the problem?   
</div>

In [ ]:
variables = [fA; fB; fC; fD; fE; fG; fH; fI; fJ; fK; fL;
             c1A; c1B; c1C; c1D; c1E; c1G; c1H; c1I; c1J; c1K; c1L;
             c2A; c2B; c2C; c2D; c2E; c2G; c2H; c2I; c2J; c2K; c2L]

values = value.(variables)

dual_c1Ma = dual(LowerBoundRef(c1M))
dual_c1Mb = dual(UpperBoundRef(c1M))
dual_c2Ma = dual(LowerBoundRef(c2M))
dual_c2Mb = dual(UpperBoundRef(c2M))
println("The dual numbers for the outlet concentrations are $(dual_c1Ma), $(dual_c1Mb), $(dual_c2Ma), $(dual_c2Mb)")

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Based on the form of the optimization problem, would you expect a local (convex) optimizer to return the best possible objective value? Why or why (not)? What sort of numerical experiments would support a determination in either case?  
</div>

<hr style="border:2px solid gray"> </hr>

## A question to reflect on

- We've solved two optimization problems: an integer program and a nonlinear program. The full design of a wastewater treatment plant may be handled by combining these two approaches. This can include which process units, how many process units, and treatment units are included as well as a full exploration of recycle loop configurations. This variety of problem is termed [**superstructure optimization**](https://www.sciencedirect.com/science/article/abs/pii/S0098135405002991) and developing efficient means of solving these problems remains an active area of research. **How might we modify the above problem using 0-1 integer variables to explore other potential recycle configurations?**